In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
fopen = open('../dataset/digit/train.csv', 'r')
train_df = pd.read_csv(fopen)
fopen.close()
fopen = open('../dataset/digit/test.csv', 'r')
test_df = pd.read_csv(fopen)
fopen.close()

In [3]:
train_df.info()
print '----------------------'
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [5]:
train_label = train_df.label 
train_df.drop(['label'], inplace=True, axis=1)

train_arr = train_df.values
test_arr = test_df.values

AttributeError: 'DataFrame' object has no attribute 'label'

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_arr, train_label, test_size=0.05, random_state=42)
n_sample = X_train.shape[0]

In [20]:
def add_layer(input, input_size, output_size, activation_func=None):
    w = tf.Variable(tf.random_normal([input_size, output_size]))
    b = tf.Variable(tf.random_normal([1, output_size]))
    u = tf.matmul(input, w) + b
    return activation_func(u) if activation_func else u


In [21]:
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])

hidden_layer = add_layer(xs, 784, 500, activation_func=tf.nn.sigmoid)
output_layer = add_layer(hidden_layer, 500, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer, ys))
opt = tf.train.AdamOptimizer(0.1).minimize(loss)

In [31]:
step = 0
y_train[step%n_sample : step%n_sample + 100]

1554     3
4273     3
8960     4
24517    3
3308     8
41181    7
35423    8
11042    8
23077    3
3016     2
11178    7
20120    6
1975     7
11546    8
32104    8
35251    7
24887    0
37541    7
40174    3
37407    1
7330     4
33825    9
32597    1
30011    1
2645     8
7940     2
3698     4
7193     1
17512    3
8426     3
        ..
31445    0
6262     3
12681    6
39201    5
4775     5
12033    6
31060    2
22519    9
9134     3
9672     5
816      7
3248     8
41547    9
18266    9
35875    5
26554    5
32435    6
26161    3
24362    1
40066    0
20777    9
34845    3
4608     0
6756     4
5925     6
1791     1
30139    3
29755    9
12928    1
21523    5
Name: label, dtype: int64

In [46]:

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for step in xrange(5000):
        data_train = X_train[step%n_sample : step%n_sample + 200, :]
        label_train = y_train[step%n_sample : step%n_sample + 200]
        sess.run(opt, feed_dict={ xs : data_train, ys : pd.get_dummies(label_train).values })
        if step % 100 == 0:
            print (step, sess.run(loss, feed_dict={ xs : data_train, ys : pd.get_dummies(label_train).values }))
    predict = sess.run(tf.arg_max(output_layer, 1), feed_dict={ xs : X_test })

(0, 22.272738)
(100, 0.15622161)
(200, 0.1491942)
(300, 0.26723871)
(400, 0.060141686)
(500, 0.29346019)
(600, 0.18936744)
(700, 0.70789093)
(800, 0.39922029)
(900, 0.30407006)
(1000, 0.19327568)
(1100, 0.43512046)
(1200, 0.24398096)
(1300, 0.5277558)
(1400, 2.100426)
(1500, 0.57479268)
(1600, 1.0389827)
(1700, 0.57147431)
(1800, 0.67697328)
(1900, 0.6864844)
(2000, 3.8736517)
(2100, 0.74553758)
(2200, 3.1921484)
(2300, 0.85208815)
(2400, 0.6343872)
(2500, 1.3404731)
(2600, 0.92087537)
(2700, 3.4219632)
(2800, 0.85318911)
(2900, 0.88505936)
(3000, 0.59374356)
(3100, 1.3189768)
(3200, 0.88713056)
(3300, 1.2119373)
(3400, 2.1113052)
(3500, 0.6766156)
(3600, 0.70722079)
(3700, 0.68960977)
(3800, 9.7360439)
(3900, 1.118916)
(4000, 0.82525516)
(4100, 1.1577903)
(4200, 2.1815917)
(4300, 1.6483033)
(4400, 0.75495821)
(4500, 1.2328075)
(4600, 0.9283278)
(4700, 1.6938574)
(4800, 1.2955245)
(4900, 0.92490387)


In [47]:
from sklearn.metrics import classification_report
print classification_report(y_test, predict)

             precision    recall  f1-score   support

          0       0.67      0.56      0.61       216
          1       0.94      0.91      0.92       234
          2       0.61      0.52      0.56       222
          3       0.63      0.49      0.55       260
          4       0.53      0.41      0.47       196
          5       0.38      0.18      0.25       160
          6       0.70      0.68      0.69       199
          7       0.67      0.34      0.45       230
          8       0.35      0.81      0.48       191
          9       0.38      0.62      0.48       192

avg / total       0.60      0.56      0.56      2100



(2100,)

In [78]:
predict = model.predict(X_test)

In [83]:
output_csv(predict, 'rf_1000.csv')

In [71]:
def output_csv(predict, file_name):
    import csv
    output = zip(range(1, test_arr.shape[0] + 1), predict)
    output_file = open(file_name, 'w')
    writer = csv.writer(output_file)
    writer.writerow(['ImageId', 'Label'])
    writer.writerows(output)
    output_file.close()